<a href="https://colab.research.google.com/github/HyeonhoonLee/KoNLP/blob/master/BERT_kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. EDA

In [211]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [212]:
import numpy as np 
import pandas as pd
import sklearn 
from sklearn import model_selection
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline

In [213]:
num_classes=26

In [214]:
DATA_IN_PATH = '/content/drive/My Drive/Colab Notebooks/meditact/'

In [215]:
print("파일 크기 : ")
for file in os.listdir(DATA_IN_PATH):
    if 'csv' in file :
        print(file.ljust(30) + str(round(os.path.getsize(DATA_IN_PATH + file) / 1000000, 2)) + 'MB')

파일 크기 : 
PRE_FINAL.csv                 1.38MB
OSAM_FINAL.csv                2.9MB


In [216]:
#loading csv data
all_data = pd.read_csv(DATA_IN_PATH + 'OSAM_FINAL.csv', quoting = 2)
all_data.head()

,symptom,class
0,질문하겠습니다. 이어폰을 음량을 0으로 하고 끼고만 있어도 귀에 안 좋나요?,ENT
1,독감예방주사,IP
2,목에 음식물 그대로 끼어서 계속 이물감이 있습니다,ENT
3,케겔운동 궁금한것,REHM
4,등 통증 관련문의,NS


In [217]:
# all_data의 전체 길이
len(all_data)

51134

In [218]:
#stratififed split to train_data, test_data

#from sklearn.model_selection import StratifiedShuffleSplit

#split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)

#for train_idx, test_idx in split.split(all_data, all_data["class"]):
#    train_data = all_data.loc[train_idx]
#    test_data = all_data.loc[test_idx]

In [219]:
# 보통은 문자열이 아닌 데이터는 삭제하나, 증상의 경우 발열, 체중 등과 같은 내용이 중요하므로 숫자도 살려둔다.
#train_symptom = [symptom for symptom in train_data['symptom'] if type(symptom) is (str or int or float)]

In [220]:
# DERM: 피부과 / GS: 외과 / IP: 호흡기내과 / FM: 가정의학과 / GI: 소화기내과 / OPH: 안과
# NR: 신경과 / ENT: 이비인후과 / PSY : 신경정신과 / HEON: 혈액종양내과 / RHEU: 류마티스내과
# REHM: 재활의학과 / NS: 신경외과 / AN: 마취통증의학과 / DENT: 치과 / PS: 성형외과
# RAD: 영상의학과 / CS: 흉부외과 / INFC: 감염내과 / RAON: 방사선종양학과 / OS: 정형외과 
# EMR: 응급의학과 / ENDO: 내분비내과 / CA: 순환기내과 / KTM: 한방과 / OBGY: 산부인과
# URO: 비뇨기과 / ALL: 알레르기 내과 / NPH: 신장내과 / OEM:직업환경의학과 / COAN: 대장항문외과
# LAB: 진단검사의학과 

In [221]:
# convert class from string to integer
class_to_label = {'DERM': 0, 'GS': 1, 'IP': 2, 'GI':3, 'OPH':4,
                  'NR': 5, 'ENT': 6, 'PSY': 7, 'HEON': 8, 'RHEU': 9,
                  'REHM': 10, 'NS': 11, 'AN': 12, 'DENT': 13, 'PS': 14,
                  'CS': 15, 'INFC': 16, 'OS': 17,
                  'EMR': 18, 'ENDO': 19, 'CA': 20, 'KTM': 21, 'OBGY': 22,
                  'URO': 23, 'ALL': 24, 'NPH': 25}
all_data['label'] = all_data['class'].map(class_to_label)
all_data.head()

,symptom,class,label
0,질문하겠습니다. 이어폰을 음량을 0으로 하고 끼고만 있어도 귀에 안 좋나요?,ENT,6
1,독감예방주사,IP,2
2,목에 음식물 그대로 끼어서 계속 이물감이 있습니다,ENT,6
3,케겔운동 궁금한것,REHM,10
4,등 통증 관련문의,NS,11


## 2. BERT를 이용한 fine tuning

In [222]:
# installing transforemrs
!pip install transformers

In [238]:
import re
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras as keras
from transformers import *

In [224]:
BATCH_SIZE = 32
NUM_EPOCHS = 10
MAX_LEN = 30 # EDA상 3사분위 수가 word수가 7이었다. token 갯수는 대략 30으로 가정해본다.
DATA_OUT_PATH = '/content/drive/My Drive/Workspace/Meditact/data/'

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt', do_lower_case=False)

### 토크나이저 테스트

In [225]:
test_sentence = "안녕하세요, 반갑습니다."

encode = tokenizer.encode(test_sentence)
token_print = [tokenizer.decode(token) for token in encode]

print(encode)
print(token_print)

[101, 9521, 118741, 35506, 24982, 48549, 117, 9321, 118610, 119081, 48345, 119, 102]
['[ C L S ]', '안', '# # 녕', '# # 하', '# # 세', '# # 요', ',', '반', '# # 갑', '# # 습', '# # 니 다', '.', '[ S E P ]']


In [226]:
# 스페셜 토큰
print(tokenizer.all_special_tokens, "\n", tokenizer.all_special_ids)

# 토크나이저 테스트하기
kor_encode = tokenizer.encode("안녕하세요, 반갑습니다. ")
eng_encode = tokenizer.encode("Hello world")

kor_decode = tokenizer.decode(kor_encode)
eng_decode = tokenizer.decode(eng_encode)

print(kor_encode)
print(eng_encode)
print(kor_decode)
print(eng_decode)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] 
 [100, 102, 0, 101, 103]
[101, 9521, 118741, 35506, 24982, 48549, 117, 9321, 118610, 119081, 48345, 119, 102]
[101, 31178, 11356, 102]
[CLS] 안녕하세요, 반갑습니다. [SEP]
[CLS] Hello world [SEP]


### Symptom to Specialty Classification (SSC)

In [227]:
# Bert Tokenizer

# 참조: https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=encode_plus#transformers.PreTrainedTokenizer.encode_plus

def bert_tokenizer(sent, MAX_LEN, truncation=True):
    
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
        max_length = MAX_LEN,           # Pad & truncate all sentences.  
        pad_to_max_length = True,
        return_attention_mask = True,
        truncation = True   # Construct attn. masks.
        
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences
    
    return input_id, attention_mask, token_type_id

In [239]:
# all

input_ids = []
attention_masks = []
token_type_ids = []
all_data_labels = []

for all_sent, all_label in tqdm(zip(all_data["symptom"], all_data["label"]), total=len(all_data)):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(all_sent, MAX_LEN)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        all_data_labels.append(all_label)

    except Exception as e:
        print(e)
        print(all_sent)
        pass

all_symptom_input_ids = np.array(input_ids, dtype=int)
all_symptom_attention_masks = np.array(attention_masks, dtype=int)
all_symptom_type_ids = np.array(token_type_ids, dtype=int)
all_symptom_inputs = (all_symptom_input_ids, all_symptom_attention_masks, all_symptom_type_ids)


all_data_labels = np.asarray(all_data_labels, dtype=np.int32)
### one-hot encoding
all_data_labels = keras.utils.to_categorical(all_data_labels)

print("# sents: {}, # labels: {}".format(len(all_symptom_input_ids), len(all_data_labels)))

  0%|          | 0/51134 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 51134/51134 [00:12<00:00, 4070.97it/s]


# sents: 51134, # labels: 51134


In [251]:
## model_selection module에 맞는 배열로 변경
arrayed_input = np.asarray(all_symptom_inputs)
transposed_input = np.transpose(arrayed_input, (1,0,2))
transposed_input.shape

(51134, 3, 30)

In [252]:
# 최대 길이: 30
input_id = all_symptom_input_ids[1]
attention_mask = all_symptom_attention_masks[1]
token_type_id = all_symptom_type_ids[1]

print(input_id)
print(attention_mask)
print(token_type_id)
print(tokenizer.decode(input_id))

[   101   9088 105197  96279  42337  16323  12945    102      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0]
[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[CLS] 독감예방주사 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [253]:
all_data_labels[1]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [291]:
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range),
                                                activation = 'softmax', ## softmax를 활성화함수로 추가함으로써 categoricalcrossentropy 계산이 가능하게 만들어줌.
                                                name="classifier")
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False): 
        ## split해서 다시 튜플로 만들기
        s0, s1, s2 = tf.split(inputs, 3, axis=1)
        inputs = all_symptom_input_ids, all_symptom_attention_masks, all_symptom_type_ids = s0[:,0,:], s1[:,0,:], s2[:,0,:]
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1] 
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

In [292]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [293]:
def create_model():
    model = TFBertClassifier(model_name='bert-base-multilingual-cased',
                                  dir_path='bert_ckpt',
                                  num_class=num_classes)
    optimizer = tf.keras.optimizers.Adam(5e-5)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    model.compile(optimizer=optimizer, loss=loss, metrics=[f1_m, precision_m, recall_m, 'AUC'], experimental_run_tf_function=False)

    return model
    

In [294]:
from sklearn import model_selection
from keras.callbacks import EarlyStopping

kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=1234)

In [296]:
fold_no = 1
f1_per_fold = []
precision_per_fold = []
recall_per_fold = []
auc_per_fold = []

es = EarlyStopping(monitor='val_loss', min_delta=0.0001,patience=2)

for train, test in kfold.split(transposed_input, all_data_labels):

    # Define the model architecture
    model = create_model()
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(transposed_input[train], all_data_labels[train],
                batch_size=100,
                epochs=30,
                verbose=1,
                validation_data = (transposed_input[test], all_data_labels[test]),
                callbacks=[es])
    scores = model.evaluate(transposed_input[test], all_data_labels[test], verbose=1)
    print(f'Score for fold {fold_no}:')
    print('loss :', scores[0])
    print('f1 score :', scores[1])
    print('precision score :', scores[2])
    print('recall score :', scores[3])
    print('AUC score :', scores[4])
    print('------------------------------------------------------------------------')

    f1_per_fold.append(scores[1])
    precision_per_fold.append(scores[2])
    recall_per_fold.append(scores[3])
    auc_per_fold.append(scores[4])
    fold_no = fold_no + 1


Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/30


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


461/461 [==============================] - ETA: 0s - loss: 2.8115 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.7631

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


...

------------------------------------------------------------------------
Training for fold 10 ...
Epoch 1/30


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


461/461 [==============================] - ETA: 0s - loss: 1.9749 - f1_m: 0.3605 - precision_m: 0.5910 - recall_m: 0.2782 - auc: 0.8817

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


461/461 [==============================] - 129s 241ms/step - loss: 1.9738 - f1_m: 0.3609 - precision_m: 0.5914 - recall_m: 0.2785 - auc: 0.8818 - val_loss: 1.0161 - val_f1_m: 0.7249 - val_precision_m: 0.7926 - val_recall_m: 0.6695 - val_auc: 0.9643
Epoch 2/30
461/461 [==============================] - 108s 233ms/step - loss: 0.9113 - f1_m: 0.7447 - precision_m: 0.8318 - recall_m: 0.6748 - auc: 0.9735 - val_loss: 0.9200 - val_f1_m: 0.7526 - val_precision_m: 0.8072 - val_recall_m: 0.7057 - val_auc: 0.9702
Epoch 3/30
461/461 [==============================] - 108s 234ms/step - loss: 0.7257 - f1_m: 0.7916 - precision_m: 0.8542 - recall_m: 0.7382 - auc: 0.9822 - val_loss: 0.8911 - val_f1_m: 0.7608 - val_precision_m: 0.8151 - val_recall_m: 0.7139 - val_auc: 0.9722
Epoch 4/30
461/461 [==============================] - 108s 233ms/step - loss: 0.5946 - f1_m: 0.8240 - precision_m: 0.8746 - recall_m: 0.7794 - auc: 0.9876 - val_loss: 0.8644 - val_f1_m: 0.7628 - val_precision_m: 0.8018 - val_recall

In [297]:
import statistics as st

print(st.mean(f1_per_fold))
print(st.mean(precision_per_fold))
print(st.mean(recall_per_fold))
print(st.mean(auc_per_fold))

0.7676456868648529
0.7987844169139862
0.7400177836418151
0.9640010595321655
